In [26]:
#from dense_unet_3d import DUnet
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm
from collections import Counter
import scipy.io as io
import statistics
import pickle

In [27]:
size1 = 128
size2 = 96

In [28]:
#################################
#Decoding layer: Same as traditional 3D Unet--upsamples total number of features except in slice directions
def decoder_layer(input_, x, ch, name):
        

    up = tf.layers.conv3d_transpose(input_,filters=12,kernel_size = [2,2,1],strides = [2,2,1],padding='SAME',name='upsample'+str(name), use_bias=False)
    up = tf.concat([up,x], axis=-1, name='merge'+str(name))
    return up
#################################


In [29]:
################################
#Encoding layer: Differences from Dense-Unet -- uses growth of 12 (ie number of channels at each convolution layer) 
#and iterates a number times and concatenates each feature map

#Currently pool size and kernal size are each set to [2x2x1] so that all features maps are halved at the height and width 
#but no the slice number--this allows for variable input sizes

# example: x_con.shape = [1,H,W,S]; after max-pooling with pool_size = [2x2x1], kernal = [2x2x1] --> x_con.shape = [1, H/2, W/2, S] 
def encoder_layer(x_con, iterations, name,training, pool=True):
    
    with tf.name_scope("encoder_block_{}".format(name)):
        for i in range(iterations):
            
            x = tf.layers.conv3d(x_con,12,kernel_size=[3,3,3],padding='SAME')
            x = tf.layers.dropout(x,rate=0.1,training=training)
            x = tf.layers.batch_normalization(x,training=training,renorm=True)
            x = tf.nn.relu(x)
            x_con = tf.concat([x,x_con], axis = 4)
        if pool is False:
            return x_con
        
        #x = tf.layers.conv3d(x_con,12,kernel_size=[1,1,1],padding='SAME')
        #x = tf.layers.dropout(x,rate=0.1,training=training)
        #x = tf.layers.batch_normalization(x,training=training,renorm=True)
        #x = tf.nn.relu(x)
        pool = tf.layers.max_pooling3d(x_con,pool_size = [2,2,1], strides=[2,2,1],data_format='channels_last')

        return x_con, pool
#################################

In [30]:
###########################################
#Dense-Unet model:  -- uses a dice loss function along w/ softmax cross entropy;  
class DUnet():
    def __init__(self, x, training):
        #self.filters = filters
        self.training = training
        self.model = self.DU_net(x)

    
    def DU_net(self,input_):


        conv1, pool1 = encoder_layer(input_,iterations=2,name="encode_"+str(1),training=self.training, pool=True)
        conv2, pool2 = encoder_layer(pool1,iterations=4,name="encode_"+str(2),training=self.training, pool=True)
        conv3, pool3 = encoder_layer(pool2,iterations=6,name="encode_"+str(3),training=self.training, pool=True)
        conv4, pool4 = encoder_layer(pool3,iterations=8,name="encode_"+str(4),training=self.training, pool=True)
        conv5, pool5 = encoder_layer(pool4,iterations=10,name="encode_"+str(5),training=self.training, pool=True)
        conv6 = encoder_layer(pool5,iterations=12,name="encode_"+str(5),training=self.training, pool=False)
        up1 = decoder_layer(conv6,conv5,10,name=1)
        conv7 = encoder_layer(up1,iterations=10,name="conv"+str(6),training=self.training, pool=False)
        up2 = decoder_layer(conv7,conv4,8,name=2)
        conv8 = encoder_layer(up2,iterations=8,name="encode_"+str(7),training=self.training, pool=False)
        up3 = decoder_layer(conv8,conv3,6,name=3)
        conv9 = encoder_layer(up3,iterations=6,name="encode_"+str(8),training=self.training, pool=False)
        up4 = decoder_layer(conv9,conv2,4,name=4)
        conv10 = encoder_layer(up4,iterations= 4,name="encode_"+str(9),training=self.training, pool=False)
        up5 = decoder_layer(conv10,conv1,2,name=5)
        conv11 = encoder_layer(up5,iterations= 2,name="encode_"+str(10),training=self.training, pool=False)

        #The final layer to generate a probability map from the CNN: channels input should be total number of classes (ie. a binary segmentation of 0 or 1 has a total number of classes = 2)
        score = tf.layers.conv3d(conv11,2,(1,1,1),name='logits',padding='SAME')

        return score

In [31]:

"""
To test dataset on the network and weights from dense_unet.py
"""


###################################
#Function to decipher tfrecord of test dataset
def feed_data(pathData):
    data_path = pathData  # path to testing tfrecord
    feature = {'test/image': tf.FixedLenFeature([], tf.string),
               'test/label': tf.FixedLenFeature([], tf.string),
               'test/depth': tf.FixedLenFeature([], tf.int64),
               'test/height': tf.FixedLenFeature([], tf.int64),
               'test/width': tf.FixedLenFeature([], tf.int64),
               'test/velocity': tf.FixedLenFeature([], tf.string) }
    
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([data_path])

    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)

    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    height = tf.cast(features["test/height"], tf.int32)
    width = tf.cast(features["test/width"], tf.int32)
    depth = tf.cast(features["test/depth"], tf.int32)

    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['test/image'], tf.float32)
    
    # Cast label data from string back to the numbers
    label = tf.decode_raw(features['test/label'], tf.float32)

    # Reshape image data into the original shape
    image = tf.reshape(image, [height, width,depth])
    label = tf.reshape(label, [height, width,depth])

    #Crop image for input size if desired
    image = tf.image.resize_image_with_crop_or_pad(image, size1,size2)
    label = tf.image.resize_image_with_crop_or_pad(label, size1, size2)
    #image = tf.image.resize_image_with_crop_or_pad(image, 128, 96)
    #label = tf.image.resize_image_with_crop_or_pad(label, 128, 96)
    #image = image[172:268]
    #label = label[172:268]
    #Cast image and label to float32
    label = tf.cast(label,tf.float32)
    image = tf.cast(image,tf.float32)

    #This hard-codes batch-size to 1 for testing--would need to rewrite input pipeline to enable batch_size > 1
    image = tf.expand_dims(image,axis = 0)
    label = tf.expand_dims(label,axis = 0)
    image = tf.expand_dims(image,axis = 4) #Add grayscale channel to input image (ie. image_shape = HxWx1 for gray or HxWx3 for color)

    image = (image - tf.reduce_min(image))/(tf.reduce_max(image) - tf.reduce_min(image)) #Normalize input image

    q = tf.FIFOQueue(capacity=50, dtypes=[tf.float32, tf.float32])# set up queue
    enqueue_op = q.enqueue_many([image,label])
    qr = tf.train.QueueRunner(q,[enqueue_op])

    return image, label, height, width 

In [32]:

#########################
#Calculate the Dice score between label and CNN output
def cost_dice(logits, labels,name='cost'):
    with tf.name_scope('cost'):
        eps = 1e-5
        #N,H,W,C,J = labels.get_shape()
        logits = tf.nn.softmax(logits)
        logits = logits[...,1]>=0.5 #used a threshold value of 0.5 for dice calculations
        #logits = logits[...,1]
        logits = tf.cast(logits,tf.float32)
        labels = labels

        log = tf.reshape(logits,[1,-1])
        
        labels = tf.reshape(labels,[1,-1])
        
        inte = tf.multiply(log,labels)
        inter = eps + tf.reduce_sum(inte)
        union =  tf.reduce_sum(log) + tf.reduce_sum(labels)+eps
        dice = tf.reduce_mean(2* inter/ (union))
        #loss = 1- loss
        return dice


In [33]:

def Unet_test(pathData,num, path, pathSave):

    #image_batch_placeholder = tf.placeholder(tf.float32, shape=[None, 128, 96,None, 1])
    #label_batch_placeholder = tf.placeholder(tf.float32, shape=[None, 128, 96,None])
    image_batch_placeholder = tf.placeholder(tf.float32, shape=[None, size1, size2,None, 1])
    label_batch_placeholder = tf.placeholder(tf.float32, shape=[None, size1, size2,None])
    
    labels_pixels = tf.reshape(label_batch_placeholder, [-1, 1])   
    training_flag = tf.placeholder(tf.bool)
    image_batch,label_batch,height, width = feed_data(pathData)


    

    logits = DUnet(x = image_batch_placeholder, training=training_flag).model #Runs CNN, logits = raw CNN output
    #logits = logits>1
    logs = cost_dice(logits,label_batch_placeholder)

    llogits = tf.nn.softmax(logits) #Apply softmax to CNN output to generate probability map
    
    

    checkpoint = tf.train.get_checkpoint_state(path) #Path to the pre-trained weights that you want to test
    saver = tf.train.Saver()


    config = tf.ConfigProto(log_device_placement=False)
    all_trainable_vars = tf.reduce_sum([tf.reduce_prod(v.shape) for v in tf.trainable_variables()])

    config.gpu_options.allow_growth=True

    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        tf.logging.info("Restoring full model from checkpoint file %s",checkpoint.model_checkpoint_path)
        saver.restore(sess, checkpoint.model_checkpoint_path)


        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord, sess = sess)
        ave = [] #To calculate average dice scores
        ma = []  #Collect a vector of all test data CNN outputs for ROC graph and AUC calculation
        tru = [] #Collect a vector of all test data labels for ROC graph and AUC calculation
        use = [] #Save a matstruct of all masks from CNN
        gtruth = [] #Save a matstruct of all labels
        low = [] #Save a matstruct of all masks from CNN w/ a dice score < 0.9
        low_true = [] #Save a matstruct of all labels w/ a dice score < 0.9
        low_loss = [] #To calculate average dice scores of datasets w/ a dice score < 0.9
        total = 15 #Number of testing datasets
        for i in tqdm(range(int(total))):
            image_out, truth = sess.run([image_batch, label_batch])
            
            _, dice, llogit = sess.run([logits, logs, llogits], feed_dict={image_batch_placeholder: image_out,
                                                                                    label_batch_placeholder: truth,
                                                                                    training_flag: True})
            

           
            infer_out = llogit[...,1] > 0.5 #CNN mask w/ a threshold of 0.5 (ie. (pixels > 0.5) = 1, else pixels = 0)
            

            O = np.float32(image_out) #Orginal image
          
            Y = np.float32(truth) #Orginal label/mask
            
            data = np.squeeze(infer_out)
            truth = np.squeeze(truth)
            dice = dice
            if dice <1.00:
                print(dice)
                print(i)
                low_d = data.transpose()
                low.append(low_d)
                low_t = truth.transpose()
                low_true.append(low_t)
                low_loss.append(dice)
            ave.append(dice)
            #print(ys.shape)
           
            #gt = np.squeeze(Y)


            mask = data.transpose()
            gtuse = truth.transpose()
            gtruth.append(gtuse)
            use.append(mask)
            #gt = gt.transpose()
            da = np.reshape(data,(-1,1))
            gt = np.reshape(truth,(-1,1))
            ma.append(da)
            tru.append(gt)
           
            
            G = (infer_out)
            #G = np.reshape(G,(256,256))
            U = np.squeeze(Y)
            W = np.squeeze(G)
            lo = np.squeeze(O)
            #ys = (ys)
            #print(W.shape)
            #plt.imshow(U)
            #plt.show()
            #plt.pause(0.1)
            #plt.imshow(W)
            #plt.show()
            #plt.pause(0.1)
            
            #plt.imshow(lo)
            #plt.show()

            #if(i == 0):
            #segmentor = "Amanda"
            #else:
            #    segmentor = "Liliana"

            #strPath = pathSave + '/mask' + segmentor + num + '.mat'
            #io.savemat(strPath,{'data':use})
            #strPath = pathSave + '/low' + segmentor + num + '.mat'
            #io.savemat(strPath,{'data':low})
            #strPath = pathSave + '/low_truth' + segmentor + num + '.mat'
            #io.savemat(strPath,{'truth':low_true})
            #strPath = pathSave + '/input_image' + segmentor + num + '.mat'
            #io.savemat(strPath,{'input':image_out})
            #strPath = pathSave+ '/gt' + segmentor + num + '.mat'
            #io.savemat(strPath,{'truth':gtruth})
        
        strPath = pathSave + '/mask' + num + '.mat'
        io.savemat(strPath,{'data':use})
        strPath = pathSave + '/low' + num + '.mat'
        io.savemat(strPath,{'data':low})
        strPath = pathSave + '/low_truth' + num + '.mat'
        io.savemat(strPath,{'truth':low_true})
        strPath = pathSave + '/input_image' + num + '.mat'
        io.savemat(strPath,{'input':image_out})
        strPath = pathSave+ '/gt'  + num + '.mat'
        io.savemat(strPath,{'truth':gtruth})

        
        
        print(sess.run(all_trainable_vars))

        print(sum(ave)/len(ave)) #Average dice scores for testing dataset

        
       
        

        #print(statistics.stdev(ave))


        

        tf.train.write_graph(sess.graph_def, 'graph/', 'my_graph.pb', as_text=False)

        coord.request_stop()
        coord.join(threads)
        sess.close()
    return 0


In [42]:
def main():
    
    for cv in range(0,5):
        num = str(cv+1)
        path = "./MagVelLoss/trainingModelCV" + str(num)
        print(path)
        
        pathData = r"D:\LA_Vel_Loss\data_formatting\test_3dCV" + str(cv+1) + ".tfrecords"
        print(pathData)
        savePath = r"D:\LA_Vel_Loss\data_results_mag_vel_loss"
        
        tf.reset_default_graph()

        Unet_test(pathData, num, path, savePath)
    

In [43]:
if __name__ == '__main__':
    main()
    print("All done :)")

./MagVelLoss/trainingModelCV1
D:\LA_Vel_Loss\data_formatting\test_3dCV1.tfrecords
INFO:tensorflow:Restoring full model from checkpoint file ./MagVelLoss/trainingModelCV1\hb.ckpt-15001
INFO:tensorflow:Restoring parameters from ./MagVelLoss/trainingModelCV1\hb.ckpt-15001


  7%|▋         | 1/15 [00:03<00:46,  3.31s/it]

0.010237512
0


 13%|█▎        | 2/15 [00:03<00:32,  2.49s/it]

0.1293233
1


...

 93%|█████████▎| 14/15 [00:13<00:00,  1.39it/s]

3.7432155e-09
13


100%|██████████| 15/15 [00:14<00:00,  1.04it/s]

0.37824827
14


6118924
0.12570784010740396
All done :)
